In [12]:
import warnings
warnings.filterwarnings("ignore")
from keras.preprocessing import text
from keras.utils import to_categorical as np_utils
from keras.preprocessing import sequence
from keras.utils import pad_sequences
import numpy as np
import pandas as pd


In [13]:
data = """Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised. 
Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks, convolutional neural networks and Transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.
"""
dl_data = data.split()

In [14]:
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(dl_data)
word2id = tokenizer.word_index

word2id['PAD'] = 0
id2word = {v:k for k, v in word2id.items()}
wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in dl_data]

vocab_size = len(word2id)
embed_size = 100
window_size = 2 

print('Vocabulary Size:', vocab_size)
print('Vocabulary Sample:', list(word2id.items())[:10])

Vocabulary Size: 75
Vocabulary Sample: [('learning', 1), ('deep', 2), ('networks', 3), ('neural', 4), ('and', 5), ('as', 6), ('of', 7), ('machine', 8), ('supervised', 9), ('have', 10)]


In [15]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

def generate_context_word_pairs(corpus, window_size, vocab_size):
    context_length = window_size * 2
    for words in corpus:
        sentence_length = len(words)
        for index, word in enumerate(words):
            context_words = []
            label_word = []

            start = index - window_size
            end = index + window_size + 1
            
            # Collect context words within the window, excluding the target word itself
            context_words.append([words[i] 
                                  for i in range(start, end) 
                                  if 0 <= i < sentence_length and i != index])
            label_word.append(word)

            # Pad sequences and convert the target word to one-hot encoding
            x = pad_sequences(context_words, maxlen=context_length)
            y = to_categorical(label_word, vocab_size)
            yield (x, y)

# Example usage
i = 0
for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
    if 0 not in x[0]:
        # Uncomment this line to print out the context and target words
        # print('Context (X):', [id2word[w] for w in x[0]], '-> Target (Y):', id2word[np.argwhere(y[0])[0][0]])
        
        if i == 10:
            break
        i += 1


In [16]:
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

cbow = Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size,)))
cbow.add(Dense(vocab_size, activation='softmax'))
cbow.compile(loss='categorical_crossentropy', optimizer='rmsprop')

print(cbow.summary())

# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 4, 100)            7500      
                                                                 
 lambda_1 (Lambda)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 75)                7575      
                                                                 
Total params: 15,075
Trainable params: 15,075
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
cbow.fit(x, y, epochs=10, verbose=1)


Epoch 1/10
1/1 [==============================] - 0s 366ms/step - loss: 4.3419
Epoch 2/10
1/1 [==============================] - 0s 12ms/step - loss: 4.2885
Epoch 3/10
1/1 [==============================] - 0s 9ms/step - loss: 4.2496
Epoch 4/10
1/1 [==============================] - 0s 14ms/step - loss: 4.2165
Epoch 5/10
1/1 [==============================] - 0s 0s/step - loss: 4.1865
Epoch 6/10
1/1 [==============================] - 0s 17ms/step - loss: 4.1584
Epoch 7/10
1/1 [==============================] - 0s 0s/step - loss: 4.1315
Epoch 8/10
1/1 [==============================] - 0s 2ms/step - loss: 4.1055
Epoch 9/10
1/1 [==============================] - 0s 0s/step - loss: 4.0801
Epoch 10/10
1/1 [==============================] - 0s 17ms/step - loss: 4.0551


In [18]:
weights = cbow.get_weights()[0]
weights = weights[1:]
print(weights.shape)
pd.DataFrame(weights, index=list(id2word.values())[1:]).head()

(74, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
deep,0.011789,-0.007853,-0.032367,0.043839,0.033197,-0.017721,-0.005434,-0.018167,0.047098,0.031254,...,0.012347,0.035228,-0.008665,0.037718,-0.028806,0.008319,-0.029723,-0.037206,-0.031099,0.023117
networks,0.025685,-0.024396,0.015882,0.036429,0.013462,-0.024105,-0.016979,0.044073,0.016253,0.026007,...,0.040271,-0.025711,0.039019,0.049549,-0.031667,-0.019136,-0.036071,-0.021611,0.040377,-0.000743
neural,0.021369,-0.037499,0.011968,-0.010972,0.010635,-0.033003,0.001413,-0.009269,-0.025414,-0.033145,...,-0.001875,0.028988,0.045560,0.027543,-0.014195,-0.021459,0.039212,0.036417,0.017058,0.047800
and,0.043388,0.014058,0.043437,0.001271,-0.003630,0.017216,-0.040695,0.000405,-0.012385,0.035985,...,0.026619,0.002998,0.048101,0.002935,-0.035240,0.031394,0.047035,-0.047138,0.013086,-0.045583
as,-0.015635,-0.019654,0.032853,0.023982,0.020615,-0.000331,0.025626,0.046809,0.033608,0.021122,...,-0.046573,-0.009891,0.037070,-0.016326,0.013357,0.042205,0.006753,-0.015512,0.034336,0.018493


In [19]:
from sklearn.metrics.pairwise import euclidean_distances

distance_matrix = euclidean_distances(weights)
print(distance_matrix.shape)

similar_words = {search_term: [id2word[idx] for idx in distance_matrix[word2id[search_term]-1].argsort()[1:6]+1] 
                   for search_term in ['deep']}

similar_words

(74, 74)


{'deep': ['as', 'unsupervised', 'family', 'surpassing', 'based']}